In [3]:
import json
with open('data/test_qa.json', 'r') as f:
    test_qa = json.load(f)

with open('data/test_sum.json', 'r') as f:
    test_sum = json.load(f)

In [2]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

model_folder = "llama2-7b-chat-ft/checkpoint-40/"

model = AutoPeftModelForCausalLM.from_pretrained(
    model_folder,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained(model_folder)

Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.51s/it]
/home/user01/naser-workspace/DIABOLOCOM/llm_ft/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/user01/naser-workspace/DIABOLOCOM/llm_ft/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding 

In [29]:
def inference(prompt, task):
    if task == 'qa':
        real = prompt['instruction'].split('### Answer:\n')[1].replace('</s>','').strip()
        text = prompt['instruction'].split('### Answer:\n')[0] + '### Answer:\n'
    elif task == 'sum':
        real = prompt['instruction'].split('### Summary:\n')[1].replace('</s>','').strip()
        text = prompt['instruction'].split('### Summary:\n')[0] + '### Summary:\n'

    input_ids = tokenizer(text, return_tensors="pt").input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, max_new_tokens=50, temperature=0.7)

    if task == 'qa':
        return {'real': real,
                'pred': 
                tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0].split('### Answer:\n')[1]
        }
    elif task == 'sum':
        return {'real': real,
                'pred':   
                tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0].split('### Summary:\n')[1]
        }

In [30]:
inference(test_sum[20], task='sum')

/home/user01/naser-workspace/DIABOLOCOM/llm_ft/venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


{'real': "Beth wants to organize a girls weekend to celebrate her mother's 40th birthday. She also wants to work at Deidre's beauty salon. Deidre offers her a few hours on Saturdays as work experience. They set up for a meeting tomorrow.",
 'pred': "Beth wants to try working in the salon for a few weeks as work experience. She'll meet Maxine tomorrow."}

In [25]:
test_qa[0]

{'instruction': '<s>### Instruction:\nYou are a helpful, respectful and honest assistant. Your task is to answer the following question. Your answer should be based on the provided context only.\n\n### Question:\nWho had an 80s No 1 hit with Hold On To The Nights?\n\n### Context:\n[DOC] [TLE] Hold On to the Nights"Hold On to the Nights" is a power ballad and number-one hit for American rock singer/songwriter/musician Richard Marx.  This was the fourth and final single released from his self-titled debut album, and the first to reach the top of the Billboard Hot 100 chart. [PAR] Written by Richard Marx, "Hold On to the Nights" reached the Billboard Hot 100 number 1 position on July 23, 1988, preventing Def Leppard\'s "Pour Some Sugar On Me" from reaching the top spot that same week. The song was on the chart for twenty-one weeks, and left the chart at number 65. From Marx\' debut 1987 album, Richard Marx, the song also reached number three on the Billboard Adult Contemporary chart. "Hol